In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
from Amex_Metric import amex_metric

## Reading data-file 
data = pd.read_csv('Delinquency_Features.csv')
data.head()

,customer_ID,target,D_39_mean,D_39_median,D_39_min,D_39_max,D_39_range,D_39_IQR,D_39_values_above_mean
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,0.010704,0.002483,0.001082,0.091492,0.090393,0.003593,1.0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0,0.215088,0.211670,0.002224,0.567383,0.564941,0.293152,5.0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0,0.004181,0.004230,0.000802,0.009705,0.008904,0.003944,7.0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0,0.048859,0.007423,0.000660,0.268555,0.267822,0.032093,3.0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0,0.004642,0.004147,0.000030,0.008682,0.008652,0.004740,6.0


In [2]:
## Defining input and target variables
X = data.drop(columns = ['customer_ID', 'target'], axis = 1)
Y = data['target']

## Spliting the data into train, validation, and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

## Defining the customized scoring function 
amex_function = make_scorer(amex_metric, greater_is_better = True, needs_proba = True)

## Defining list to store results
features_to_select = list()

for i in range(0, 2):

    ## Running RFE with Random forest
    RF_auto_feature = RFECV(estimator = RandomForestClassifier(n_estimators = 50, max_depth = 5), step = 1, scoring = amex_function, min_features_to_select = 5, cv = 3).fit(X_train, Y_train)
    
    ## Appending results 
    features_to_select.append(X_train.columns[RF_auto_feature.support_])
    
print(features_to_select)

[Index(['D_39_mean', 'D_39_median', 'D_39_max', 'D_39_range', 'D_39_IQR',
       'D_39_values_above_mean'],
      dtype='object'), Index(['D_39_mean', 'D_39_median', 'D_39_max', 'D_39_range', 'D_39_IQR',
       'D_39_values_above_mean'],
      dtype='object')]


In [5]:
pd.DataFrame(features_to_select)

,0,1,2,3,4,5
0,D_39_mean,D_39_median,D_39_max,D_39_range,D_39_IQR,D_39_values_above_mean
1,D_39_mean,D_39_median,D_39_max,D_39_range,D_39_IQR,D_39_values_above_mean


In [6]:
features_to_select.append(['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7'])

In [8]:
pd.DataFrame(features_to_select)

,0,1,2,3,4,5,6
0,D_39_mean,D_39_median,D_39_max,D_39_range,D_39_IQR,D_39_values_above_mean,None
1,D_39_mean,D_39_median,D_39_max,D_39_range,D_39_IQR,D_39_values_above_mean,None
2,X_1,X_2,X_3,X_4,X_5,X_6,X_7


In [12]:
RF_auto_feature.ranking_

array([1, 1, 2, 1, 1, 1, 1])

In [13]:
RF_auto_feature.support_

array([ True,  True, False,  True,  True,  True,  True])

In [14]:
RF_auto_feature

RFECV(cv=3, estimator=RandomForestClassifier(max_depth=5, n_estimators=50),
      min_features_to_select=5,
      scoring=make_scorer(amex_metric, needs_proba=True))

In [8]:
## Identifying important features 
print(X_train.columns[RF_auto_feature.support_])

Index(['D_39_mean', 'D_39_median', 'D_39_min', 'D_39_max', 'D_39_range',
       'D_39_IQR', 'D_39_values_above_mean'],
      dtype='object')


In [15]:
## Identifying important features 
print(X_train.columns[RF_auto_feature.support_])

Index(['D_39_mean', 'D_39_median', 'D_39_max', 'D_39_range', 'D_39_IQR',
       'D_39_values_above_mean'],
      dtype='object')


In [10]:
X_train.shape

(367130, 7)

In [ ]:
## Defining input and target variables
X = data.drop(columns = ['customer_ID', 'target'], axis = 1)
Y = data['target']

## Spliting the data into train, validation, and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

## Defining hyper-paramerters for RF
RF_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15],
                 'min_samples_leaf': [5, 7],
                 'max_depth' : [3, 5, 7]}

## Defining the customized scoring function 
amex_function = make_scorer(amex_metric, greater_is_better = True, needs_proba = True)

## Performing grid search
RF_grid_search = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = amex_function, n_jobs = -1).fit(X_train, Y_train)

# Feature Selecting with Imputed Variables

In [14]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 500) 
import numpy as np

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

## Defining files names
file_key = 'AmericanExpress/Delinquency_Features_Filled.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading data-files
data = pd.read_csv(file_content_stream)
data = data.drop(columns = ['D_64_last'], axis = 1)

## Putting variables in the right shape 
data['D_68_last'] = data['D_68_last'].astype(str)
data['D_114_last'] = data['D_114_last'].astype(str)
data['D_116_last'] = data['D_116_last'].astype(str)
data['D_117_last'] = data['D_117_last'].astype(str)
data['D_120_last'] = data['D_120_last'].astype(str)
data['D_126_last'] = data['D_126_last'].astype(str)

## Converting to dummies
dummies = pd.get_dummies(data[['D_63_last', 'D_68_last', 'D_114_last', 'D_116_last', 'D_117_last', 'D_120_last', 'D_126_last']])

## Appeding dummies 
data = data.drop(columns = ['D_63_last', 'D_68_last', 'D_114_last', 'D_116_last', 'D_117_last', 'D_120_last', 'D_126_last'], axis = 1)
data = pd.concat([data, dummies], axis = 0)
data.head()

,customer_ID,target,D_39_mean,D_39_median,D_39_min,D_39_max,D_39_range,D_39_IQR,D_39_std,D_39_pct_values_above_mean,D_41_mean,D_41_median,D_41_min,D_41_max,D_41_range,D_41_IQR,D_41_std,D_41_pct_values_above_mean,D_44_mean,D_44_median,D_44_min,D_44_max,D_44_range,D_44_IQR,D_44_std,D_44_pct_values_above_mean,D_45_range,D_45_pct_values_above_mean,D_47_mean,D_47_median,D_47_min,D_47_max,D_47_range,D_47_IQR,D_47_std,D_47_pct_values_above_mean,D_47_avg_pct_change,D_51_mean,D_51_median,D_51_min,D_51_max,D_51_range,D_51_IQR,D_51_std,D_51_pct_values_above_mean,D_52_mean,D_52_median,D_52_min,D_52_max,D_52_range,D_52_IQR,D_52_std,D_52_pct_values_above_mean,D_52_avg_pct_change,D_54_mean,D_54_median,D_54_min,D_54_max,D_54_range,D_54_IQR,D_54_std,D_54_pct_values_above_mean,D_54_avg_pct_change,D_55_pct_values_above_mean,D_58_mean,D_58_median,D_58_min,D_58_max,D_58_range,D_58_IQR,D_58_std,D_58_pct_values_above_mean,D_59_mean,D_59_median,D_59_min,D_59_max,D_59_range,D_59_IQR,D_59_std,D_59_pct_values_above_mean,D_59_avg_pct_change,D_60_mean,D_60_median,D_60_min,D_60_max,D_60_range,D_60_IQR,D_60_std,D_60_pct_values_above_mean,D_65_mean,D_65_median,D_65_min,D_65_max,D_65_range,D_65_IQR,D_65_pct_values_above_mean,D_68_pct_values_above_mean,D_69_median,D_69_min,D_69_max,D_69_range,D_69_IQR,D_69_pct_values_above_mean,D_70_mean,D_70_median,D_70_min,D_70_max,D_70_range,D_70_IQR,D_70_std,D_70_pct_values_above_mean,D_71_mean,D_71_median,D_71_min,D_71_max,D_71_range,D_71_IQR,D_71_std,D_71_pct_values_above_mean,D_72_mean,D_72_median,D_72_min,D_72_max,D_72_range,D_72_IQR,D_72_std,D_72_pct_values_above_mean,D_74_mean,D_74_median,D_74_min,D_74_max,D_74_range,D_74_IQR,D_74_std,D_74_pct_values_above_mean,D_75_mean,D_75_median,D_75_min,D_75_max,D_75_range,D_75_IQR,D_75_std,D_75_pct_values_above_mean,D_78_mean,D_78_median,D_78_min,D_78_max,D_78_range,D_78_IQR,D_78_std,D_78_pct_values_above_mean,D_79_mean,D_79_median,D_79_min,D_79_max,D_79_range,D_79_IQR,D_79_std,D_79_pct_values_above_mean,D_80_mean,D_80_median,D_80_min,D_80_max,D_80_range,D_80_IQR,D_80_std,D_80_pct_values_above_mean,D_81_mean,D_81_median,D_81_min,D_81_max,D_81_range,D_81_IQR,D_81_std,D_81_pct_values_above_mean,D_83_mean,D_83_median,D_83_min,D_83_max,D_83_range,D_83_IQR,D_83_std,D_83_pct_values_above_mean,D_84_mean,D_84_median,D_84_min,D_84_max,D_84_range,D_84_IQR,D_84_std,D_84_pct_values_above_mean,D_86_mean,D_86_median,D_86_min,D_86_max,D_86_range,D_86_IQR,D_86_std,D_86_pct_values_above_mean,D_89_mean,D_89_median,D_89_min,D_89_max,D_89_range,D_89_IQR,D_89_std,D_89_pct_values_above_mean,D_91_mean,D_91_median,D_91_min,D_91_max,D_91_range,D_91_IQR,D_91_std,D_91_pct_values_above_mean,D_92_mean,D_92_median,D_92_min,D_92_max,D_92_range,D_92_IQR,D_92_std,D_92_pct_values_above_mean,D_93_mean,D_93_median,D_93_min,D_93_max,D_93_range,D_93_IQR,D_93_std,D_93_pct_values_above_mean,D_94_mean,D_94_median,D_94_min,D_94_max,D_94_range,D_94_IQR,D_94_std,D_94_pct_values_above_mean,D_96_mean,D_96_median,D_96_min,D_96_max,D_96_range,D_96_IQR,D_96_std,D_96_pct_values_above_mean,D_102_mean,D_102_median,D_102_min,D_102_max,D_102_range,D_102_IQR,D_102_std,D_102_pct_values_above_mean,D_103_mean,D_103_median,D_103_min,D_103_max,D_103_range,D_103_IQR,D_103_std,D_103_pct_values_above_mean,D_104_mean,D_104_median,D_104_min,D_104_max,D_104_range,D_104_IQR,D_104_std,D_104_pct_values_above_mean,D_107_mean,D_107_median,D_107_min,D_107_max,D_107_range,D_107_IQR,D_107_std,D_107_pct_values_above_mean,D_109_mean,D_109_median,D_109_min,D_109_max,D_109_range,D_109_IQR,D_109_std,D_109_pct_values_above_mean,D_112_mean,D_112_median,D_112_min,D_112_max,D_112_range,D_112_IQR,D_112_std,D_112_pct_values_above_mean,D_112_avg_pct_change,D_113_mean,D_113_median,D_113_min,D_113_max,D_113_range,D_113_IQR,D_113_std,D_113_pct_values_above_mean,D_115_mean,D_115_median,D_115_min,D_115_max,D_115_range,D_115_IQR,D_115_std,D_115_pct_values_above_mean,D_115_avg_pct_change,D_118_mean,D_118_median,D_118_min,D_118_max,D_118_range,D_118_IQR,D_118_std,D_118_

In [15]:
data.shape

(458913, 476)

In [21]:
## Putting variables in the right shape 
data['D_68_last'] = data['D_68_last'].astype(str)
data['D_114_last'] = data['D_114_last'].astype(str)
data['D_116_last'] = data['D_116_last'].astype(str)
data['D_117_last'] = data['D_117_last'].astype(str)
data['D_120_last'] = data['D_120_last'].astype(str)
data['D_126_last'] = data['D_126_last'].astype(str)

In [22]:
X = pd.get_dummies(data[['D_63_last', 'D_68_last', 'D_114_last', 'D_116_last', 'D_117_last', 'D_120_last', 'D_126_last']])
X

,D_63_last_CL,D_63_last_CO,D_63_last_CR,D_63_last_XL,D_63_last_XM,D_63_last_XZ,D_68_last_1.0,D_68_last_2.0,D_68_last_3.0,D_68_last_4.0,D_68_last_5.0,D_68_last_6.0,D_114_last_0.0,D_114_last_1.0,D_116_last_0.0,D_116_last_1.0,D_117_last_-1.0,D_117_last_1.0,D_117_last_2.0,D_117_last_3.0,D_117_last_4.0,D_117_last_5.0,D_117_last_6.0,D_120_last_0.0,D_120_last_1.0,D_126_last_0.0,D_126_last_1.0
0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1
2,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1
3,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1
4,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,1
458909,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1
458910,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1
458911,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1


In [20]:
pd.get_dummies(data['D_68_last'].astype(str))

,1.0,2.0,3.0,4.0,5.0,6.0
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,1,0,0,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
458908,0,0,0,0,0,1
458909,0,0,0,0,0,1
458910,0,0,0,0,1,0
458911,0,0,1,0,0,0
